In [1]:
import pandas as pd
%run heartbeat_slicer.ipynb

In [2]:
import os
import numpy as np
from matplotlib import pyplot as plt
import gudhi
import gudhi.representations.vector_methods as grvm
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_point_cloud

In [3]:
folder_path = "data_processed/mitdb"
record_names_path = "data_original/mitdb/RECORDS"

with open(record_names_path, 'r') as record_names_file:
    record_names = record_names_file.readlines()

record_names = [record_name[:-1] for record_name in record_names]

In [4]:
dataMitDB = []

for record_name in record_names:
    sliced = HeartbeatSlicer(folder_path + '/' + record_name + '_processed')
    dataMitDB.append(sliced.annotate_heartbeats())

In [5]:
#[patient][heartbeat_num] - [0]:heartbeat [1]:ann
dataMitDB[2][3][0].shape

(497,)

In [68]:
def make_point_cloud(heartbeat_):

    embedding_dimension_periodic = 3
    embedding_time_delay_periodic = 8
    embedder_periodic = SingleTakensEmbedding(
        parameters_type="fixed",
        n_jobs=2,
        time_delay=embedding_time_delay_periodic,
        dimension=embedding_dimension_periodic,
    )
    point_cloud_ = embedder_periodic.fit_transform(heartbeat)
    plot_point_cloud(point_cloud_)

    return point_cloud_


In [87]:
def make_complex_w_pers_diag(point_cloud_):
    point_cloud_ = point_cloud_[None, :, :]

    homology_dimensions = [0, 1]

    periodic_persistence = VietorisRipsPersistence(
        homology_dimensions=homology_dimensions, n_jobs=-1
    )

    diag_ = periodic_persistence.fit_transform(point_cloud_)

    return diag_


In [88]:
def make_betti_curve(diag_):
    bc = BettiCurve(n_jobs=-1)
    betti_curve_ = bc.fit_transform(diag_)
    return betti_curve_

In [89]:
# betti = make_betti_curve(dataMitDB[2][2][0])

In [90]:
# betti[0][0]

In [ ]:
df_to_be = []

for patient in dataMitDB:
    for heartbeat, ann in patient:
        # Convert the segment to a point cloud
        point_cloud = make_point_cloud(heartbeat)

        # Compute the Vietoris-Rips complex and its persistence diagram
        diag = make_complex_w_pers_diag(point_cloud)

        # Compute and return Betti curve
        betti = make_betti_curve(diag)

        df_to_be.append([point_cloud, diag, betti, ann])

df = pd.DataFrame(df_to_be, columns=['points_cloud', 'diag', 'betti', 'ann'])

In [85]:
df


""


In [53]:
make_point_cloud(dataMitDB[0][0][0])

array([[-0.01220739,  0.08545174,  0.00500303],
       [-0.01600969,  0.13508179, -0.01460885],
       [-0.02121284,  0.1721042 , -0.02181321],
       ...,
       [ 0.0104063 ,  0.00900545,  0.00440267],
       [ 0.0104063 ,  0.00860521,  0.00360218],
       [ 0.01020618,  0.00820497,  0.0028017 ]])

In [ ]:
make_point_cloud()